In [1]:
import pandas as pd
import numpy as np
data = pd.read_pickle('dt_trxpse_V3.pkl.gzip')

In [2]:
data.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp
0,230435642,3,2122392.51,CC,NaN,NaN,NaN,NaN,NaN,2016-12-07 11:34:51,cc,vacio,vacio
1,222356110,10,148438.37,Referencia: Contrato: Valor:,CC,NaN,NaN,NaN,NaN,2016-10-16 00:34:24,"[referencia, contrato, valor]",cc,cc
2,309137749,10,94025.19,CC,NaN,NaN,NaN,NaN,NaN,2018-01-20 19:50:42,cc,vacio,vacio
3,324614737,10,94430.07,CC,NaN,NaN,NaN,NaN,NaN,2018-03-26 19:21:46,cc,vacio,vacio
4,235344690,18,670645.57,MEDICINA PREPAGADA COLSANITAS,CE,NaN,NaN,NaN,NaN,2017-01-06 20:13:17,"[medicina, prepagada, colsanitas]",ce,ce


In [3]:
#Obtiene las categorías de sectores y subsectores en las que el cliente del registro más 
#gastó (hasta un máximo de max_categorias) en la ventana de tiempo comprendida
#entre fecha_base y ventana_dias días antes
#row: pandas.Series con información de una fila de un dataframe
#ventana_dias_int: Cantidad de días anteriores a la fecha del registro desde donde se
#empezará a hacer la jerarquización de las categorías
#df: DataFrame en donde se hará la búsqueda de los registros para poder hacer la jerarquización.
def top_categories_single(row,ventana_dias_int,df, max_categorias=3):
    ventana_dias = np.timedelta64(ventana_dias_int,'D')
    id = row['id_cliente']
    fecha_base =row['fecha_usable']
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<fecha_base)]
    top_categorias = transacciones_cliente['sector'].value_counts().index#.groupby('sector')['sector'].value_counts().index
    top_subcategorias = transacciones_cliente['subsector'].value_counts().index#.groupby('subsector')['subsector'].value_counts().index
    cant_top_categorias = len(top_categorias)
    cant_top_subcategorias = len(top_subcategorias)
    return pd.Series([top_categorias[i] if i < cant_top_categorias else np.nan for i in range(max_categorias)] +
                     [top_subcategorias[i] if i < cant_top_subcategorias else np.nan for i in range(max_categorias)])

#Obtiene las categorías de sectores y subsectores en las que el cliente con id_cliente id más 
#gastó (hasta un máximo de max_categorias) en la ventana de tiempo comprendida
#entre fecha_base y ventana_dias días antes.

def top_categories(df,id, fecha_base, ventana_dias, max_categorias=5):
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)]
    top_categorias = transacciones_cliente.groupby('sector')['sector'].count()
    top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()

    if len(top_subcategorias)>max_categorias:
        top_categorias = top_categorias[0:max_categorias-1]
        top_subcategorias = top_subcategorias[0:max_categorias-1]
    
    
    return top_categorias.index.get_values(), top_subcategorias.index.get_values()

#Obtiene el total gastado por el cliente con id_cliente id en lo que va del mes de fecha_base.
def total_consumo(df,id, fecha_base):
    ventana = fecha_actual-fecha_actual.astype('datetime64[M]')
    transacciones_cliente = df[df.id_cliente == id]
    transacciones_cliente = transacciones_cliente[(transacciones_cliente.fecha_usable>=(fecha_base-ventana_dias)) & (transacciones_cliente.fecha_usable<=fecha_base)]
    conusmo_mes = transacciones_cliente['valor_trx'].sum()
    #top_subcategorias = transacciones_cliente.groupby('subsector')['subsector'].count()
    return conusmo_mes

In [8]:
data_test = data[:100]

In [9]:
%timeit data_test.apply(top_categories_single,axis=1,ventana_dias_int=30,df=data)

1min 2s ± 358 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
#ejemplo de uso:
max_category_columns = data_test.apply(top_categories_single,axis=1,ventana_dias_int=30,df=data_test)
max_category_columns.columns = ['max_sector_1','max_sector_2','max_sector_3','max_subsector_1','max_subsector_2','max_subsector_3']
#después hacer un join
data_test.join(max_category_columns)

KeyboardInterrupt: 

In [ ]:
#ejemplo de uso:
max_category_columns = test_df.apply(top_categories_single,axis=1,ventana_dias_int=30,df=small_df)
max_category_columns.columns = ['max_sector_1','max_sector_2','max_sector_3','max_subsector_1','max_subsector_2','max_subsector_3']
#después hacer un join
test_df.join(max_category_columns)

In [ ]:
#Otra solución en la que estoy trabajando. Podría ser más rápida
#small_df.groupby([]).size().unstack(fill_value=0)
#r =test_df.groupby(['id_cliente'],as_index=False).resample('D',on='fecha_usable')#['sector'].count()#['sector']#.size().unstack(fill_value=0)#.groupby(['id_cliente']).count()#.size().unstack(fill_value=0)
sized_df=small_df.groupby(['id_cliente',pd.Grouper(key='fecha_usable',freq='D'),'sector'],as_index=False).size().unstack(fill_value=0).sort_index(level=['id_cliente','fecha_usable'])#.resample('D',level='fecha_usable').sum()

In [ ]:
sized_df['GOBIERNO']

In [ ]:
sized_df.rolling(3,min_periods=0).sum()['GOBIERNO']

In [ ]:
##preparación de datos para prueba
id = '26729'
fecha_actual = np.datetime64('2018-04-20')
ventana_dias = 10

In [ ]:
sector, subsector = top_categories(small_df,id,fecha_actual,10)
print(sector)
print(subsector)

In [ ]:
#scraper páginas amarillas
